In [8]:
import re
import os
from datetime import datetime

def extract_tables(sql_script):
    """
    Extract table names and identify volatile tables from the SQL script using regex.
    """
    create_table_regex = re.compile(r'CREATE\s+(VOLATILE\s+)?TABLE\s+(\w+)', re.IGNORECASE)
    other_table_regex = re.compile(r'(INSERT\s+INTO|MERGE\s+INTO|JOIN|FROM|UPDATE)\s+(\w+)', re.IGNORECASE)

    all_tables = set()
    volatile_tables = set()

    # Find tables created with CREATE TABLE statement
    create_matches = create_table_regex.findall(sql_script)
    for match in create_matches:
        table_name = match[1]
        all_tables.add(table_name)
        if match[0].strip().upper() == 'VOLATILE':
            volatile_tables.add(table_name)

    # Find tables mentioned with other SQL keywords
    other_matches = other_table_regex.findall(sql_script)
    for match in other_matches:
        table_name = match[1]
        if table_name not in volatile_tables:
            all_tables.add(table_name)

    return list(all_tables), list(volatile_tables)

def generate_statements(tables, with_data=True):
    """
    Generate SQL statements to create test tables using Teradata SQL syntax.
    """
    statements = []
    current_date = datetime.now().strftime('%d_%m_%Y')
    for table in tables:
        test_table = f"{table}_test_{current_date}"
        if with_data:
            statement = f"CREATE TABLE {test_table} AS {table} WITH DATA;"
        else:
            statement = f"CREATE TABLE {test_table} AS {table} WITH NO DATA;"
        statements.append(statement)
    return statements

def create_test_tables(sql_script, with_data=True):
    """
    Automate the process of creating test tables.
    """
    # Extract table names and volatile tables from the SQL script
    all_tables, volatile_tables = extract_tables(sql_script)
    
    # Identify non-volatile tables
    non_volatile_tables = [table for table in all_tables if table not in volatile_tables]
    
    # Generate SQL statements for test tables
    test_statements = generate_statements(non_volatile_tables, with_data)
    
    return test_statements

def process_files(directory, with_data=True):
    """
    Process all SQL files in the given directory.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".sql"):
            input_file_path = os.path.join(directory, filename)
            output_file_path = os.path.join(directory, f"test_{filename}")
            
            # Read the SQL script from the input file
            with open(input_file_path, 'r') as file:
                sql_script = file.read()
            
            # Create test table statements
            test_statements = create_test_tables(sql_script, with_data)
            
            # Write the test table statements to the output file
            with open(output_file_path, 'w') as file:
                for statement in test_statements:
                    file.write(statement + '\n')
            
            print(f"Processed {filename} and saved test table SQL statements to test_{filename}")

if __name__ == "__main__":
    # Path to the directory containing SQL files
    directory_path = r"C:\Users\asus\OneDrive\Desktop\sql_scripts"

    # Ask the user if they want the tables with data or without data
    with_data_input = input("Do you want to create the tables with data? (yes/no): ").strip().lower()
    with_data = with_data_input in ['yes', 'y']

    # Process all SQL files in the directory
    process_files(directory_path, with_data)


Do you want to create the tables with data? (yes/no): no
Processed enablers_lottie.sql and saved test table SQL statements to test_enablers_lottie.sql
Processed example1.sql and saved test table SQL statements to test_example1.sql
Processed example2.sql and saved test table SQL statements to test_example2.sql
Processed example3.sql and saved test table SQL statements to test_example3.sql
Processed example4.sql and saved test table SQL statements to test_example4.sql


In [ ]:
import re
import os
from datetime import datetime

def extract_tables(sql_script):
    """
    Extract table names and identify volatile tables from the SQL script using regex.
    """
    create_table_regex = re.compile(r'CREATE\s+(VOLATILE\s+)?TABLE\s+(\w+)', re.IGNORECASE)
    other_table_regex = re.compile(r'(INSERT\s+INTO|MERGE\s+INTO|JOIN|FROM|UPDATE)\s+(\w+)', re.IGNORECASE)

    all_tables = set()
    volatile_tables = set()

    # Find tables created with CREATE TABLE statement
    create_matches = create_table_regex.findall(sql_script)
    for match in create_matches:
        table_name = match[1]
        all_tables.add(table_name)
        if match[0].strip().upper() == 'VOLATILE':
            volatile_tables.add(table_name)

    # Find tables mentioned with other SQL keywords
    other_matches = other_table_regex.findall(sql_script)
    for match in other_matches:
        table_name = match[1]
        if table_name not in volatile_tables:
            all_tables.add(table_name)

    return list(all_tables), list(volatile_tables)

def generate_statements(tables, with_data=True):
    """
    Generate SQL statements to create test tables using Teradata SQL syntax.
    """
    statements = []
    current_date = datetime.now().strftime('%d_%m_%Y')
    for table in tables:
        test_table = f"{table}_test_{current_date}"
        if with_data:
            statement = f"CREATE TABLE {test_table} AS {table} WITH DATA;"
        else:
            statement = f"CREATE TABLE {test_table} AS {table} WITH NO DATA;"
        statements.append(statement)
    return statements

def create_test_tables(sql_script, with_data=True):
    """
    Automate the process of creating test tables.
    """
    # Extract table names and volatile tables from the SQL script
    all_tables, volatile_tables = extract_tables(sql_script)
    
    # Identify non-volatile tables
    non_volatile_tables = [table for table in all_tables if table not in volatile_tables]
    
    # Generate SQL statements for test tables
    test_statements = generate_statements(non_volatile_tables, with_data)
    
    return test_statements

def process_files(directory, print_to_single_file=True, with_data=True):
    """
    Process all SQL files in the given directory.
    """
    all_statements = []

    for filename in os.listdir(directory):
        if filename.endswith(".sql"):
            input_file_path = os.path.join(directory, filename)
            
            # Read the SQL script from the input file
            with open(input_file_path, 'r') as file:
                sql_script = file.read()
            
            # Create test table statements
            test_statements = create_test_tables(sql_script, with_data)

            # Accumulate statements
            all_statements.extend(test_statements)

            if not print_to_single_file:
                # Write statements to a separate file
                output_file_path = os.path.join(directory, f"test_{filename}")
                with open(output_file_path, 'w') as file:
                    for statement in test_statements:
                        file.write(statement + '\n')

                print(f"Processed {filename} and saved test table SQL statements to {output_file_path}")

    if print_to_single_file:
        # Write all statements to a single output file
        output_file_path = os.path.join(directory, "test_tables.sql")
        with open(output_file_path, 'w') as file:
            for statement in all_statements:
                file.write(statement + '\n')

        print(f"Processed all SQL files and saved test table SQL statements to {output_file_path}")

if __name__ == "__main__":
    # Path to the directory containing SQL files
    directory_path = r"C:\Users\asus\OneDrive\Desktop\sql_scripts"

    # Ask the user if they want the tables with data or without data
    with_data_input = input("Do you want to create the tables with data? (yes/no): ").strip().lower()
    with_data = with_data_input in ['yes', 'y']

    # Ask the user if they want to print all files to a single file or separate files
    print_to_single_file_input = input("Do you want to print the SQL output to a single file? (yes/no): ").strip().lower()
    print_to_single_file = print_to_single_file_input in ['yes', 'y']

    # Process all SQL files in the directory
    process_files(directory_path, print_to_single_file, with_data)


In [ ]:
import os
import re
import datetime
import sys

def extract_table_names(sql_script):
    """Extract table names from the SQL script."""
    table_names = set()
    # Regex pattern to find table names
    patterns = [
        r'\bCREATE\s+TABLE\s+([a-zA-Z_][\w]*)\b',
        r'\bINSERT\s+INTO\s+([a-zA-Z_][\w]*)\b',
        r'\bUPDATE\s+([a-zA-Z_][\w]*)\b',
        r'\bMERGE\s+INTO\s+([a-zA-Z_][\w]*)\b',
        r'\bJOIN\s+([a-zA-Z_][\w]*)\b'
    ]
    
    # Remove comments from the SQL script
    sql_script = re.sub(r'--.*\n', '', sql_script)  # Remove single-line comments
    sql_script = re.sub(r'/\*.*?\*/', '', sql_script, flags=re.DOTALL)  # Remove multi-line comments

    for pattern in patterns:
        matches = re.findall(pattern, sql_script, re.IGNORECASE)
        table_names.update(matches)

    return table_names

def generate_test_table_sql(table_name, with_data=True):
    """Generate the SQL statement for creating a test table."""
    current_date = datetime.datetime.now().strftime("%d_%m_%Y")
    test_table_name = f"{table_name}_test_{current_date}"
    if with_data:
        return f"CREATE TABLE {test_table_name} AS {table_name} WITH DATA;\n"
    else:
        return f"CREATE TABLE {test_table_name} AS {table_name} WITH NO DATA;\n"

def process_sql_file(file_path, with_data=True):
    """Process a single SQL file to generate test table creation statements."""
    with open(file_path, 'r') as file:
        sql_script = file.read()
    
    table_names = extract_table_names(sql_script)
    test_table_sql_statements = [generate_test_table_sql(table, with_data) for table in table_names]
    
    return test_table_sql_statements

def process_sql_directory(directory_path, output_file, with_data=True):
    """Process all SQL files in a directory."""
    all_test_table_sql_statements = []
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".sql"):
            file_path = os.path.join(directory_path, filename)
            test_table_sql_statements = process_sql_file(file_path, with_data)
            all_test_table_sql_statements.extend(test_table_sql_statements)
    
    with open(output_file, 'w') as file:
        file.writelines(all_test_table_sql_statements)

def main(args):
    import argparse

    parser = argparse.ArgumentParser(description="Generate test tables for SQL scripts.")
    parser.add_argument("directory", help="Directory containing SQL scripts")
    parser.add_argument("--output", default="test_tables.sql", help="Output file for test table creation statements")
    parser.add_argument("--with_data", action="store_true", help="Include data in test tables")
    parser.add_argument("--separate_files", action="store_true", help="Generate separate files for each script")

    # Parse only known arguments to avoid issues in interactive environments
    args, unknown = parser.parse_known_args(args)

    if args.separate_files:
        for filename in os.listdir(args.directory):
            if filename.endswith(".sql"):
                file_path = os.path.join(args.directory, filename)
                test_table_sql_statements = process_sql_file(file_path, args.with_data)
                output_file = os.path.join(args.directory, f"test_{filename}")
                with open(output_file, 'w') as file:
                    file.writelines(test_table_sql_statements)
    else:
        process_sql_directory(args.directory, args.output, args.with_data)

if __name__ == "__main__":
    main(sys.argv[1:])


In [ ]:
import re
import os
from datetime import datetime

def extract_tables(sql_script):
    """
    Extract table names and identify volatile tables from the SQL script using regex.
    """
    create_table_regex = re.compile(r'CREATE\s+(VOLATILE\s+)?TABLE\s+(\w+)', re.IGNORECASE)
    other_table_regex = re.compile(r'(INSERT\s+INTO|MERGE\s+INTO|JOIN|FROM|UPDATE)\s+(\w+)', re.IGNORECASE)

    all_tables = set()
    volatile_tables = set()

    # Find tables created with CREATE TABLE statement
    create_matches = create_table_regex.findall(sql_script)
    for match in create_matches:
        table_name = match[1]
        all_tables.add(table_name)
        if match[0].strip().upper() == 'VOLATILE':
            volatile_tables.add(table_name)

    # Find tables mentioned with other SQL keywords
    other_matches = other_table_regex.findall(sql_script)
    for match in other_matches:
        table_name = match[1]
        if table_name not in volatile_tables:
            all_tables.add(table_name)

    return list(all_tables), list(volatile_tables)

def generate_statements(tables, with_data=True):
    """
    Generate SQL statements to create test tables using Teradata SQL syntax.
    """
    statements = []
    current_date = datetime.now().strftime('%d_%m_%Y')
    for table in tables:
        test_table = f"{table}_test_{current_date}"
        if with_data:
            statement = f"CREATE TABLE {test_table} AS {table} WITH DATA;"
        else:
            statement = f"CREATE TABLE {test_table} AS {table} WITH NO DATA;"
        statements.append(statement)
    return statements

def create_test_tables(sql_script, with_data=True):
    """
    Automate the process of creating test tables.
    """
    # Extract table names and volatile tables from the SQL script
    all_tables, volatile_tables = extract_tables(sql_script)
    
    # Identify non-volatile tables
    non_volatile_tables = [table for table in all_tables if table not in volatile_tables]
    
    # Generate SQL statements for test tables
    test_statements = generate_statements(non_volatile_tables, with_data)
    
    return test_statements

def process_files(directory, with_data=True, table_name=None):
    """
    Process all SQL files in the given directory and only create test tables for scripts containing the specified table name.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".sql"):
            input_file_path = os.path.join(directory, filename)
            
            # Read the SQL script from the input file
            with open(input_file_path, 'r') as file:
                sql_script = file.read()
            
            # Check if the SQL script contains the specified table name (if provided)
            if table_name and re.search(rf'\b{table_name}\b', sql_script, re.IGNORECASE):
                output_file_path = os.path.join(directory, f"test_{filename}")
                
                # Create test table statements
                test_statements = create_test_tables(sql_script, with_data)
                
                # Write the test table statements to the output file
                with ope
